In [ ]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.preprocessing import OneHotEncoder

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## 0. Read Data

Fetal Health Classification Dataset: https://www.kaggle.com/datasets/andrewmvd/fetal-health-classification

Original article: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6822315/

In [ ]:
df_input = pd.read_csv(os.path.join(os.getcwd(), '..', 'data', 'fetal_health.csv'))
df_input.columns = [col.replace('_', ' ') for col in df_input.columns]
df_input

In [ ]:
df_input.describe().T

Fetal health (target column):
- 1 - Normal
- 2 - Suspect
- 3 - Pathological

For this notebook, we will consider only the Normal/not Normal distinction (binary classification)

In [ ]:
df_input['fetal health'].value_counts()

In [ ]:
df_input['fetal health binary'] = (df_input['fetal health'] == 1).astype(int)

In [ ]:
df_input['fetal health binary'].value_counts()

In [ ]:
X = df_input[[col for col in df_input.columns if col not in ['fetal health', 'fetal health binary']]]
y = df_input['fetal health binary']

In [ ]:
classes_dict = {0:'Not Normal', 1:'Normal'}

## 1. EDA

Illustrative EDA example

In [ ]:
plt.figure(figsize=(24, 24))

plot_cols = [col for col in X.columns if not col.startswith('histogram')]
pairplot = sns.pairplot(pd.concat([X[plot_cols], y], axis=1), hue='fetal health binary')
# rotate y labels
for ax in pairplot.axes.flatten():
    # rotate x axis labels
    ax.set_xlabel(ax.get_xlabel(), rotation=0)
    # rotate y axis labels
    ax.set_ylabel(ax.get_ylabel(), rotation=80)
    # set y labels alignment
    ax.yaxis.get_label().set_horizontalalignment('right')

plt.show()

In [ ]:
def plot_boxplots(X, y, features_to_plot):
    
    fig, axs = plt.subplots(math.ceil(len(features_to_plot)/PLOTS_PER_ROW),PLOTS_PER_ROW, figsize=(22, 4*math.ceil(len(features_to_plot)/3)))
    fig.subplots_adjust(hspace=.3, wspace=.2)
    
    i, j = 0, 0
    PLOTS_PER_ROW = 4
    for feature in features_to_plot:
        sns.boxplot(
            x=y.apply(lambda cell: classes_dict[cell]), y=X[feature], order=classes_dict.values(), ax=axs[i][j], palette="YlGnBu",
            medianprops=dict(linewidth=4, color="blue", alpha=1.0), flierprops=dict(markerfacecolor="#707070", marker="d"),
            showmeans=True, meanprops={"marker":"X","markerfacecolor":"limegreen", "markeredgecolor":"green" ,"markersize":10}
        )
        axs[i][j].set_title(feature, fontsize=MEDIUM_SIZE)
        axs[i][j].set_ylabel('', fontsize=SMALL_SIZE)
        axs[i][j].set_xlabel('', fontsize=SMALL_SIZE)

        j += 1
        if j % PLOTS_PER_ROW == 0:
            i += 1
            j = 0
            
    plt.show()

In [ ]:
plot_boxplots(X, y, X.columns)

## 2. Correlation Analysis

### 2.1. Pearson's Correlation Coefficient

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Compute features' correlation
X_corr = X.corr()

# Generate a mask to onlyshow the bottom triangle
mask_corr = np.triu(np.ones_like(X_corr, dtype=bool))

In [ ]:
plt.figure(figsize=(16,12))
plt.title("Features' Pearson's Correlation Coefficient", fontsize=18)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# generate heatmap
sns.heatmap(X_corr, cmap="YlGnBu", annot=True, mask=mask_corr, vmin=-1, vmax=1, square=True, annot_kws={"fontsize":8}, fmt=".2f")

plt.show()

Most features do not appear to have strong positive or negative correlations with other features, but there are cases where correlation is high

### 2.2. Multicollinearity

> Multicollinearity is a problem because it undermines the statistical significance of an independent variable. [\[source\]](https://link.springer.com/chapter/10.1007/978-0-585-25657-3_37)

> Multicollinearity does not affect the accuracy of predictive models, including regression models. \[...\] Now, where multicollinearity becomes 'an issue' is when you want to 'interpret' the parameters learned by your model. In other words, you cannot say that the feature with the 'biggest weight' is 'the most important' when the features are correlated. Note that this is independent on the accuracy of the model, this is only the interpretation part [\[source\]](https://www.researchgate.net/post/Are-Random-Forests-affected-by-multi-collinearity-between-features)

> Multicollinearity is only a problem for inference in statistics and analysis. For example, if you’d like to infer the importance of certain features, then almost by definition multicollinearity means that some features are shown as strongly/perfectly correlated with other combination of features, and therefore they are undistinguishable. In this case, you can simply remove the problematic features.
Multicollinearity is not a real problem for prediction. GBT, being more of a black-box model, is more suitable for predictions problems to start with, although of course you could try and use them for inference as well.
[\[source\]](https://www.quora.com/Is-multicollinearity-a-problem-with-gradient-boosted-trees)



#### 2.2.1. Variance Inflation Factor (VIF)

Here’s the formula for calculating the VIF for feature X1: $VIF_{1}=\frac{1}{1-R_{1}^{2}}$

$R^2$ in this formula is the coefficient of determination from the linear regression model which has: X1 as dependent variable (target), and X2, X3, ... as independent variables (features). To calculate VIF for each features, you have to fit a linear regression using such feature as the target.

As a general rule of thumb, "VIF > 5 is cause for concern and VIF > 10 indicates a serious collinearity problem."

- If feature X1 has VIF = 1 (minimum possible value for VIF), then there is zero collinearity between this feature and the other features in the dataset (e.g. X2, X3, ...)
- If feature X1 has VIF = 2.5, then the variance of the regression coefficient of X1 in the original linear regression model is 2.5 times greater than it would have been if X1 had been entirely non-related to other features
- If feature X1 has VIF = Inf, then X1 can be perfectly predicted by using the other features in the dataset
[\[source\]](https://quantifyinghealth.com/vif-threshold/#:~:text=Most%20research%20papers%20consider%20a,of%205%20or%20even%202.5.)

The higher the VIF:
- The more correlated a predictor is with the other predictors
- The more the standard error is inflated
- The larger the confidence interval
- The less likely it is that a coefficient will be evaluated as statistically significant
[\[source\]](https://towardsdatascience.com/everything-you-need-to-know-about-multicollinearity-2f21f082d6dc)

For implementation example with evaluation on performance and feature importance, read [\[this article\]](https://towardsdatascience.com/targeting-multicollinearity-with-python-3bd3b4088d0b)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
df_vif = pd.DataFrame(X.columns, columns=['feature'])
df_vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
df_vif = df_vif.sort_values('VIF', ascending=False)
df_vif

**Which features would remain if we removed all features with VIF > 10?**

In [ ]:
def check_vif(X, threshhold=10):
    features = list(X.columns)
    max_len = max([len(f) for f in features])
    high_vif_cols = []

    for i in range(len(features)):
        vif = [variance_inflation_factor(X[features].values, ix) for ix in range(X[features].shape[1])]
        max_idx = np.argmax(vif)
        if max(vif) > threshhold:
            high_vif_col = features.pop(max_idx)
            high_vif_cols.append(high_vif_col)
            print(f'{i+1}. Removed feature: {(high_vif_col+" ").ljust(max_len+2,".")} VIF: {vif[max_idx]:.2f}')

    print(f'\nRemaining features:\n{features}')
    
    return high_vif_cols

In [ ]:
high_vif_cols = check_vif(X)

__________

#### 2.2.2. Principal Component Analysis (PCA)

Principal Components Analysis (PCA) is a well-known unsupervised dimensionality reduction technique that constructs relevant features/variables through linear (linear PCA) or non-linear (kernel PCA) combinations of the original variables (features).

The construction of relevant features is achieved by linearly transforming correlated variables into a smaller number of uncorrelated variables. This is done by projecting (dot product) the original data into the reduced PCA space using the eigenvectors of the covariance/correlation matrix aka the principal components (PCs).

The resulting projected data are essentially linear combinations of the original data capturing most of the variance in the data.

[\[source\]](https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e)

Additional insighful article on multicollinearity and model interpretability: [\[source\]](https://marinawyss.github.io/multicollinearity/)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Standardize X
stdscaler = StandardScaler()
X_standardized = pd.DataFrame(stdscaler.fit_transform(X), columns=X.columns)

In [ ]:
# use all possible PC's
pca_initial = PCA(n_components=X_standardized.shape[1])

pca_cols = [f'PC{i}' for i in range(1,X_standardized.shape[1]+1)]
# The PCA model requires standardized (z-scored) data
X_pca = pd.DataFrame(pca_initial.fit_transform(X_standardized), columns=pca_cols)

In [ ]:
df_vif_pca = pd.DataFrame(pca_cols, columns=['PC'])
df_vif_pca['VIF'] = [variance_inflation_factor(X_pca, i) for i in range(X_pca.shape[1])]
df_vif_pca

In [ ]:
df_vif_pca['VIF'].round(decimals=1).value_counts()

With PCA, we removed all the multicolinearity in the data

In [ ]:
# Compute features' correlation
X_corr_pca = X_pca.corr()

# Generate a mask to onlyshow the bottom triangle
mask_corr_pca = np.triu(np.ones_like(X_corr_pca, dtype=bool))

In [ ]:
plt.figure(figsize=(16,12))
plt.title("Features' Pearson's Correlation Coefficient", fontsize=18)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# generate heatmap
sns.heatmap(X_corr_pca, cmap="YlGnBu", annot=True, mask=mask_corr_pca, vmin=-1, vmax=1, square=True, annot_kws={"fontsize":8}, fmt=".2f")

plt.show()

#### 2.2.3. PCA after train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(X, y, df_input.index, test_size=0.2, random_state=0)

In [ ]:
y_train.value_counts()

In [ ]:
# Standardize X_train
stdscaler = StandardScaler()
X_train_std = pd.DataFrame(stdscaler.fit_transform(X_train), columns=X.columns, index=idx_train)
X_test_std = pd.DataFrame(stdscaler.transform(X_test), columns=X.columns, index=idx_test)

In [ ]:
# A) Select components that explain 90% of the variability in the original data
n_components = np.argmax(pca_initial.explained_variance_ratio_.cumsum() > 0.98) + 1
# B) use all PC's
# n_components = X.shape[1]

# use selected PC's
pca = PCA(n_components=n_components)

pca_cols = [f'PC{i}' for i in range(1,n_components+1)]
# The PCA model requires standardized (z-scored) data
X_train_pca = pd.DataFrame(pca.fit_transform(X_train_std), columns=pca_cols, index=idx_train)
X_test_pca = pd.DataFrame(pca.transform(X_test_std), columns=pca_cols, index=idx_test)
X_train_pca_coeffs = pd.DataFrame(pca.components_, index=pca_cols, columns=X.columns)

print(f"Number of Principal Components (PC's) used: {len(pca.explained_variance_ratio_)} (out of {X.shape[1]} possible PC's)")
print(f"Those {len(pca.explained_variance_ratio_)} PC's explain {pca.explained_variance_ratio_.sum()*100:.1f}% of the variance in the original data")

## Auxiliar Functions' Definitions

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [ ]:
def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=None, normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix
    
    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(7, 5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=BIGGER_SIZE, pad=20)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, ['\n'.join(name.rsplit(' ')) for name in target_names], rotation=0, fontsize=MEDIUM_SIZE)
        plt.yticks(tick_marks, ['\n'.join(name.rsplit(' ')) for name in target_names], fontsize=MEDIUM_SIZE)
        
    cm_copy = cm.copy()
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm_copy.max() / 1.5 if normalize else cm_copy.max() / 2
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.1f}%\n({})".format(cm[i, j]*100, cm_copy[i, j]),
                     horizontalalignment="center",
                     color="white" if cm_copy[i, j] > thresh else "black",
                     fontsize=MEDIUM_SIZE)
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm_copy[i, j] > thresh else "black",
                     fontsize=MEDIUM_SIZE)

    plt.tight_layout()
    plt.ylabel('True label', fontsize=MEDIUM_SIZE, labelpad=10)
    plt.xlabel('Predicted label\n\naccuracy={:.2f}%; misclassification={:.2f}%'.format(100*accuracy, 100*misclass), fontsize=MEDIUM_SIZE, labelpad=15)
    plt.show()

In [ ]:
def gridsearch_cv(X_train_, y_train_, model, space, n_splits, n_repeats,
                  scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'f1'], refit='f1', random_state=0):
    # ignore warnings via env var because GridSearchCV with n_jobs=-1 triggers parallel backend warnings
    os.environ['PYTHONWARNINGS'] = 'ignore'
    # define evaluation
    cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
    # define search
    search = GridSearchCV(model, space, scoring=scoring, n_jobs=-1, cv=cv, refit=refit)
    # execute search
    result_cv = search.fit(X_train_, y_train_)
    
    return result_cv

Function to calculate p-values for scikit learn Logistic Regression model.

Source: https://stackoverflow.com/a/47079198

In [ ]:
# Source: https://stackoverflow.com/questions/25122999/scikit-learn-how-to-check-coefficients-significance/47079198#47079198
from scipy.stats import norm

def logit_stderror(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se
    p = (1 - norm.cdf(abs(t))) * 2
    
    return se

In [ ]:
import matplotlib.patches as mpatches

def plot_importance(df_importance, title, error=None):

    
    df_importance['Absolute Coefficients'] = df_importance['Coefficients'].abs()
    df_importance = df_importance.sort_values('Absolute Coefficients', ascending=True)

    colors_dict = {
        'Positive': 'royalblue',
        'Negative': 'firebrick'
    }
    df_importance['Color'] = df_importance.apply(
        lambda row: colors_dict['Negative'] if row['Coefficients'] < 0 else colors_dict['Positive'], axis=1
    )
    
    fig, ax = plt.subplots()
    
    max_coeff = df_importance['Absolute Coefficients'].max()
    err_max_coeff = error[np.argmax(df_importance['Absolute Coefficients'].max())]
    big_errors = []
    for i, err in enumerate(error):
        if err > max_coeff + err_max_coeff:
            big_errors.append((i, err))
            error[i] = max_coeff
            
    for idx_err, big_err in big_errors:
        display_string = f"  ...  (Margin of Error: {big_err:,.2f})"
        ax.text(df_importance['Absolute Coefficients'].iloc[idx_err] + max_coeff, idx_err-0.025, display_string, color='black', fontweight='bold')

    df_importance['Absolute Coefficients'].plot(
        kind="barh", color=df_importance['Color'], figsize=(10, df_importance.shape[0]/2), legend=False, ax=ax, 
        xerr=error, ecolor='black', error_kw={'label':'95% confidence interval', 'capsize':4, 'capthick':1}
    )
    
    ax.xaxis.grid()
    ax.set_axisbelow(True)
    plt.title(title, pad=20, fontsize=BIGGER_SIZE)
    legend_patches = [
        ax.get_legend_handles_labels()[0][0], # confidence interval
        mpatches.Patch(color=colors_dict['Positive'], label='Positive Coefficient'),
        mpatches.Patch(color=colors_dict['Negative'], label='Negative Coefficient'),
    ]
    plt.legend(handles=legend_patches, loc='lower right')
    
    plt.show()
    
        
    
    return df_importance[['Coefficients', 'Absolute Coefficients']]

In [ ]:
def print_metrics_cv(result_cv):
    print('Grid Search CV Best Model - Scoring Metrics:\n')
    print(f"ROC AUC:   {result_cv.cv_results_['mean_test_roc_auc'][result_cv.best_index_]:.3f}")
    print(f"Accuracy:  {result_cv.cv_results_['mean_test_accuracy'][result_cv.best_index_]:.3f}")
    print(f"Precision: {result_cv.cv_results_['mean_test_precision'][result_cv.best_index_]:.3f}")
    print(f"Recall:    {result_cv.cv_results_['mean_test_recall'][result_cv.best_index_]:.3f}")
    print(f"F1 score:  {result_cv.cv_results_['mean_test_f1'][result_cv.best_index_]:.3f}\n")

    print(f'\nBest Hyperparameters: {result_cv.best_params_}\n\n')

In [ ]:
def print_metrics(y_test_, y_pred_):
    print('Final Model - Scoring Metrics on Test Dataset:\n')
    print(f"ROC AUC:   {roc_auc_score(y_test_, y_pred_):.3f}")
    print(f"Accuracy:  {accuracy_score(y_test_, y_pred_):.3f}")
    print(f"Precision: {precision_score(y_test_, y_pred_):.3f}")
    print(f"Recall:    {recall_score(y_test_, y_pred_):.3f}")
    print(f"F1 score:  {f1_score(y_test_, y_pred_):.3f}\n\n")
    print('Classification Report: \n')
    print(classification_report(y_test_, y_pred_))

In [ ]:
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families

def box_tidwell_test(X, continuous_features):

    cols_to_keep = []
    X_copy = X.copy()

    # Add logit transform interaction terms (natural log) for continuous variables e.g.. Age * Log(Age)
    for var in continuous_features:
        log_var = f'log({var})'
        cols_to_keep.append(var)
        cols_to_keep.append(log_var)
        X_copy[log_var] = X_copy[var].apply(lambda x: x * np.log(x))

    # keep only numeric columns and their corresponding log columns
    X_copy = X_copy[cols_to_keep]
    X_copy_cte = sm.add_constant(X_copy, prepend=False)


    # Building model and fit the data (using statsmodel's Logit)
    logit_results = GLM(y, X_copy_cte, family=families.Binomial()).fit()

    # Display summary results
    # print(logit_results.summary())

    results_as_html = logit_results.summary().tables[1].as_html()
    df_summary = pd.read_html(results_as_html, header=0, index_col=0)[0]

    features_with_nonlinearity = []
    for idx, row in df_summary.iterrows():
        if idx[:3] == 'log' and row['P>|z|'] <= 0.05:
            features_with_nonlinearity.append(idx.split(')')[0].split('(')[1])

    print('Features with non-linear relashionship with the log-odds,')
    print('i.e. log(feature) is statistically significant (p-value<0.05):\n')
    for i, feat in enumerate(features_with_nonlinearity):
        print(f' {i+1}. {feat}')
    print('\nConsider performing higher-order polynomial transformations to capture the non-linearity (e.g., feature²).')
    
    return features_with_nonlinearity

In [ ]:
import math

def plot_log_odds_graphs(X, features_to_plot):
    
    # Add constant
    X_cte = sm.add_constant(X, prepend=False)
    # Re-run logistic regression on original set of X and y variables
    logit_results = GLM(y, X_cte, family=families.Binomial()).fit()
    predicted = logit_results.predict(X_cte)

    # Get log odds values
    log_odds = np.log(predicted / (1 - predicted))
    
    i, j = 0, 0
    PLOTS_PER_ROW = 4
    fig, axs = plt.subplots(math.ceil(len(features_to_plot)/PLOTS_PER_ROW),PLOTS_PER_ROW, figsize=(22, 5*math.ceil(len(features_to_plot)/3)))
    fig.subplots_adjust(hspace=.2, wspace=.15)
    for feature in features_to_plot:
        axs[i][j].scatter(x=X_cte[feature].values, y=log_odds, s=3)
        axs[i][j].set_xlabel(feature)
        if j == 0:
            axs[i][j].set_ylabel('Log-odds')
        j += 1
        if j % PLOTS_PER_ROW == 0:
            i += 1
            j = 0
            
    plt.show()

## 3. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
y_train.value_counts()

In [ ]:
space_logreg = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty': ['none', 'l1', 'l2', 'elasticnet'],
    'C': np.logspace(-5, 2, num=15, base=10.0), # 10e-5 to 100 in 15 steps
    'class_weight': ['balanced']
}

### Check for Linearity between independent variables and the log-odds of the output

The logistic regression model assumes a linear relationship between each independent variable and the logit (aka log-odds) of the outcome.

Explanation of Logistic Regression assumptions: https://towardsdatascience.com/assumptions-of-logistic-regression-clearly-explained-44d85a22b290

**A) Box-Tidwell Test** (only works with rows with positive values)

In [ ]:
# features_with_nonlinearity = box_tidwell_test(X, X.columns)

**B) Visual Inspection**

In [ ]:
plot_log_odds_graphs(X, X.columns)

### 3.1. Logistic Regression on PC's

In [ ]:
%%time
# Hyperparameter tuning with Grid Search Cross Validation

result_cv_logreg_pca = gridsearch_cv(X_train_pca, y_train, model=LogisticRegression(), 
                                 space=space_logreg, n_splits=4, n_repeats=2, refit='f1', random_state=0)
result_cv_logreg_pca

In [ ]:
print_metrics_cv(result_cv_logreg_pca)

In [ ]:
# define best model
model_logreg_pca = LogisticRegression(
    **result_cv_logreg_pca.best_params_, random_state=0
)

In [ ]:
# Fit model and make predictions
model_logreg_pca.fit(X_train_pca, y_train)
y_pred_pca = model_logreg_pca.predict(X_test_pca)
y_pred_proba_pca = model_logreg_pca.predict_proba(X_test_pca)
print(f'Accuracy on Training Set: {100*model_logreg_pca.score(X_train_pca, y_train):.1f}%')
print(f'Accuracy on Test Set:     {100*model_logreg_pca.score(X_test_pca, y_test):.1f}%')

In [ ]:
print_metrics(y_test, y_pred_pca)

In [ ]:
# Confusion Matrix
cm_pca = confusion_matrix(y_test, y_pred_pca)
plot_confusion_matrix(cm_pca, target_names=[classes_dict[i] for i in model_logreg_pca.classes_])

In [ ]:
print(classification_report(y_test, y_pred_pca))

In [ ]:
df_importance_components_pca = pd.DataFrame(np.transpose(model_logreg_pca.coef_), columns=['Coefficients'], index=pca_cols)
stderror_components = logit_stderror(model_logreg_pca, X_train_pca)[1:]
df_importance_components_pca = plot_importance(df_importance_components_pca, title="Logistic Regression on PC's - Absolute Coefficient Values with 95% CI", 
                                               error=1.96*stderror_components)

CI for the coefficients of the original features is the square root of the sum of the squares of the radius of the CI for the coefficients of the principal components.

Sources:
- Stack Exchange: https://stats.stackexchange.com/a/224760
- Uncertainties and Error Propagation: https://www.geol.lsu.edu/jlorenzo/geophysics/uncertainties/Uncertaintiespart2.html
- Error Propagation (Propagation of Uncertainty): https://www.statisticshowto.com/statistics-basics/error-propagation/

In [ ]:
df_importance_pca = X_train_pca_coeffs.multiply(
                        df_importance_components_pca['Coefficients'], axis='index'
                    ).sum(axis=0).to_frame(name='Coefficients')

# Source: https://stats.stackexchange.com/questions/223924/how-to-add-up-partial-confidence-intervals-to-create-a-total-confidence-interval
stderror_pca = np.sqrt((X_train_pca_coeffs.multiply(stderror_components, axis='index')**2).sum(axis=0))

df_importance_pca = plot_importance(df_importance_pca, title="Logistic Regression on PC's - Absolute Coefficient Values with 95% CI",
                                    error=1.96*stderror_pca)

### 3.2. Logistic Regression on original dataset

In [ ]:
# remove high-VIF columns

# X_train_std = X_train_std[[col for col in X_train_std.columns if col not in high_vif_cols]]
# X_test_std = X_test_std[[col for col in X_test_std.columns if col not in high_vif_cols]]

In [ ]:
%%time
# Hyperparameter tuning with Grid Search Cross Validation

result_cv_logreg = gridsearch_cv(X_train_std, y_train, model=LogisticRegression(), 
                                 space=space_logreg, n_splits=4, n_repeats=2, refit='f1', random_state=0)
result_cv_logreg

In [ ]:
print_metrics_cv(result_cv_logreg)

In [ ]:
# define best model
model_logreg = LogisticRegression(
    **result_cv_logreg.best_params_, random_state=0
)

In [ ]:
# Fit model and make predictions
model_logreg.fit(X_train_std, y_train)
y_pred = model_logreg.predict(X_test_std)
y_pred_proba = model_logreg.predict_proba(X_test_std)
print(f'Accuracy on Training Set: {100*model_logreg.score(X_train_std, y_train):.1f}%')
print(f'Accuracy on Test Set:     {100*model_logreg.score(X_test_std, y_test):.1f}%')

In [ ]:
print_metrics(y_test, y_pred)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cm, target_names=[classes_dict[i] for i in model_logreg.classes_])

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
df_importance = pd.DataFrame(np.transpose(model_logreg.coef_), columns=['Coefficients'], index=X_train_std.columns)
stderror = logit_stderror(model_logreg, X_train_std)[1:]
df_importance = plot_importance(df_importance, title="Logistic Regression - Absolute Coefficient Values with 95% CI",
                                error=1.96*stderror)

## 4. XGBoost Classifier

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
import shap

In [ ]:
y_train.value_counts()

In [ ]:
neg_class_odds = y_train[y_train == 0].count() / y_train[y_train == 1].count()
neg_class_odds = neg_class_odds.iloc[0]
neg_class_odds

XGBoost parameter explanation: https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning/notebook

In [ ]:
space_xgb = {
    'objective': ['binary:logistic'],
    'n_estimators': [40, 50, 60],
    'learning_rate': [0.1],
    'max_depth': [4, 5, 6],
    'min_child_weight': [2, 3, 4],
    'gamma': [0, 0.25, 0.5],
    'alpha':[0, 0.15, 0.3],
    # ratio of number of negative class to the positive class (sum(negative instances) / sum(positive instances))
    'scale_pos_weight': [neg_class_odds],
    'eval_metric': ['logloss'],
    'lambda':[1, 1.25],
    ## 'subsample': [0.8, 1.0],
    ## 'colsample_bytree': [0.8, 1.0],
}

### 4.1. XGBoost on PC's

In [ ]:
%%time
# Hyperparameter tuning with Grid Search Cross Validation

result_cv_xgb_pca = gridsearch_cv(X_train_pca, y_train, model=XGBClassifier(), 
                                 space=space_xgb, n_splits=4, n_repeats=1, refit='f1', random_state=0)
result_cv_xgb_pca

In [ ]:
print_metrics_cv(result_cv_xgb_pca)

In [ ]:
# define best model
model_xgb_pca = XGBClassifier(
    **result_cv_xgb_pca.best_params_, random_state=0
)

In [ ]:
# Fit model and make predictions
model_xgb_pca.fit(X_train_pca, y_train)
y_pred_pca = model_xgb_pca.predict(X_test_pca)
y_pred_proba_pca = model_xgb_pca.predict_proba(X_test_pca)
print(f'Accuracy on Training Set: {100*model_xgb_pca.score(X_train_pca, y_train):.1f}%')
print(f'Accuracy on Test Set:     {100*model_xgb_pca.score(X_test_pca, y_test):.1f}%')

In [ ]:
print_metrics(y_test, y_pred_pca)

In [ ]:
# Confusion Matrix
cm_pca = confusion_matrix(y_test, y_pred_pca)
plot_confusion_matrix(cm_pca, target_names=[classes_dict[i] for i in model_xgb_pca.classes_])

**SHAP Analysis**

In [ ]:
explainer = shap.TreeExplainer(model_xgb_pca)
shap_values_pca = explainer.shap_values(X_test_pca)

In [ ]:
plt.title('Feature Importance', fontsize=BIGGER_SIZE)
shap.summary_plot(shap_values_pca, X_test_pca, plot_type="bar", class_names=[classes_dict[i] for i in model_xgb_pca.classes_],
                  max_display=X_test_pca.shape[1], plot_size=(8,X_test_pca.shape[1]/2.5))

In [ ]:
df_shap_pca = pd.DataFrame(shap_values_pca, columns=X_test_pca.columns)
df_importance_pca = df_shap_pca.T.abs().mean(axis=1).to_frame(name='mean(|SHAP|)')

# df_importance_pca.sort_values('mean(|SHAP|)', ascending=False)

In [ ]:
shap_values = np.dot(shap_values_pca, X_train_pca_coeffs.values)

In [ ]:
df_shap = pd.DataFrame(shap_values, columns=X.columns)
df_importance = df_shap.T.abs().mean(axis=1).to_frame(name='Feature Importance - derived from mean(|SHAP|)')

# df_importance.sort_values('Feature Importance - derived from mean(|SHAP|)', ascending=False)

In [ ]:
plt.title("Feature Importance - derived from PC's mean(|SHAP|)", fontsize=BIGGER_SIZE)
shap.summary_plot(shap_values, X_test_std, plot_type="bar", class_names=[classes_dict[i] for i in model_xgb_pca.classes_], 
                  max_display=X_test_std.shape[1], plot_size=(12,X_test_std.shape[1]/2.5))

In [ ]:
shap.summary_plot(np.array(shap_values_pca), X_test_pca, feature_names=pca_cols, show=True, max_display=15, plot_size=(12,7))

In [ ]:
shap.summary_plot(np.array(shap_values), X_test_std, feature_names=X_test_std.columns, show=True, max_display=None, plot_size=(12,8))

**Feature Importance from XGBoost 'gain' metric**

In [ ]:
df_importance_gain_pca = pd.DataFrame(model_xgb_pca.feature_importances_, index=pca_cols, columns=['PC Feature Gain'])
df_importance_gain = X_train_pca_coeffs.multiply(df_importance_gain_pca['PC Feature Gain'], axis='index').sum(axis=0).abs().to_frame(name='Feature Gain')

df_importance_gain.sort_values('Feature Gain', ascending=True).plot(kind="barh", figsize=(12, X_test_std.shape[1]/2.5), legend=False)
plt.gca().xaxis.grid(True)
plt.gca().set_axisbelow(True)
plt.title("XGBoost - Original Feature's Importance\n(derived from XGBoost 'gain' importance metric)", pad=20, fontsize=BIGGER_SIZE)
plt.show()

### 4.2. XGBoost on original dataset

In [ ]:
# remove high-VIF columns

# X_train_std = X_train_std[[col for col in X_train_std.columns if col not in high_vif_cols]]
# X_test_std = X_test_std[[col for col in X_test_std.columns if col not in high_vif_cols]]

In [ ]:
%%time
# Hyperparameter tuning with Grid Search Cross Validation

result_cv_xgb = gridsearch_cv(X_train_std, y_train, model=XGBClassifier(), 
                                 space=space_xgb, n_splits=4, n_repeats=1, refit='f1', random_state=0)
result_cv_xgb

In [ ]:
print_metrics_cv(result_cv_xgb)

In [ ]:
# define best model
model_xgb = XGBClassifier(
    **result_cv_xgb.best_params_, random_state=0
)

In [ ]:
# Fit model and make predictions
model_xgb.fit(X_train_std, y_train)
y_pred = model_xgb.predict(X_test_std)
y_pred_proba = model_xgb.predict_proba(X_test_std)
print(f'Accuracy on Training Set: {100*model_xgb.score(X_train_std, y_train):.1f}%')
print(f'Accuracy on Test Set:     {100*model_xgb.score(X_test_std, y_test):.1f}%')

In [ ]:
print_metrics(y_test, y_pred)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cm, target_names=[classes_dict[i] for i in model_xgb.classes_])

**SHAP Analysis**

In [ ]:
explainer = shap.TreeExplainer(model_xgb)
shap_values = explainer.shap_values(X_test_std)

In [ ]:
plt.title('Feature Importance', fontsize=BIGGER_SIZE)
shap.summary_plot(shap_values, X_test_std, plot_type="bar", class_names=[classes_dict[i] for i in model_xgb_pca.classes_], 
                  max_display=X_test_std.shape[1], plot_size=(12,X_test_std.shape[1]/2.5))

In [ ]:
df_shap = pd.DataFrame(shap_values, columns=X_test_std.columns)
df_importance = df_shap.T.abs().mean(axis=1).to_frame(name='mean(|SHAP|)')

# df_importance.sort_values('mean(|SHAP|)', ascending=False)

In [ ]:
shap.summary_plot(np.array(shap_values), X_test_std, feature_names=X_test_std.columns, show=True, max_display=X_test_std.shape[1], plot_size=(12,8))

**Feature Importance from XGBoost 'gain' metric**

In [ ]:
df_importance_gain = pd.DataFrame(model_xgb.feature_importances_, index=X_test_std.columns, columns=['Feature Gain'])

df_importance_gain.sort_values('Feature Gain', ascending=True).plot(kind="barh", figsize=(12, X_test_std.shape[1]/2.5), legend=False)
plt.gca().xaxis.grid(True)
plt.gca().set_axisbelow(True)
plt.title("XGBoost - Feature Importance\n(derived from XGBoost 'gain' importance metric)", pad=20, fontsize=BIGGER_SIZE)
plt.show()